# Initialize

In [1]:
from sentence_transformers import SentenceTransformer, util
#import numpy as np

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Data Preprocessing Pipeline

In [4]:
file_path = "C:/Users/USER/Desktop/초록테크/"

In [49]:
ah1_blood = pd.read_csv(file_path + "ah1/Test.csv")
ah2_blood = pd.read_csv(file_path + "ah2/labresult.csv")
ah2_test = pd.read_csv(file_path + "ah2/interpretation.csv", encoding='cp949')

In [10]:
# 1. ah1

ah1_df = pd.read_csv(file_path + "ah1/EMR.csv")

ah1_df

,chiefComplaint,diagnosisDateTime,PetDiagnosisImage,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText
0,NaN,2022-02-07T10:27:01.920Z,[],[],[],[],[],1,알수없음,UNKNOWN,0.00,NaN
1,애드보킷 구매,2022-02-07T10:26:23.647Z,[],[],[],[],['BE4.[Pack]애드보킷-CAT 4-8kg'],297,신구름,CM,5.30,NaN
2,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.91,<p>S</p>\n<p>건강했음.</p>\n<p>어제 침을 너무 많이 흘린 것 이외...
3,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.60,<p>S</p>\n<p>- 특이사항 없었음</p>\n<p>O</p>\n<p>- 오른...
4,재진,2022-02-07T08:31:09.877Z,[],['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.38,<p>S</p>\n<p>더 나아지지는 않는 것 같다고 말씀</p>\n<p>활력은 양...
...,...,...,...,...,...,...,...,...,...,...,...,...
23257,NaN,2024-05-17T03:20:39.751Z,[],[],[],"['[수액] N/S, H/S 수액 처치_100ml', '[치과] 치석제거, 스켈링 ...",[],568,상추,CM,4.35,NaN
23258,재진 구토,2024-05-17T02:05:34.182Z,[],"['Metoclopramide, 메토클로프라마이드', 'Metronidazole, ...",['포티플로라 1파우치'],"['특수주사_세레니아(항구토제)_10kg 이하', '방사선 복부 2컷 10kg 미만...",[],2045,몽실,SF,5.50,<p>S</p><p>노란색 거품토를 어제 4번 하고 오늘 아침에 새벽 6시에 한번 ...
23259,상담,2024-05-17T02:01:27.360Z,[],[],[],['[진료비] 상담비(동물 미동반)'],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",2052,루,CM,6.30,<p>미동반으로 건강검진 상담위해 내원하심.</p><p>: standard + 췌장...
23260,NaN,2024-05-17T01:51:48.524Z,[],[],[],['힐스 독 z/d 1.5kg'],[],1739,탄,CM,7.70,NaN


In [4]:
len(ah1_df['Pet.id'].unique())

3419

In [16]:
valve_patient = open(file_path + "ah1/심장병환자 리스트.txt", mode='r', encoding='utf-8')
valve_patient_list = valve_patient.readlines()

for i in range(len(valve_patient_list)):
    valve_patient_list[i] = valve_patient_list[i].replace("\n", "")

for i in range(1, len(valve_patient_list)):
    valve_patient_list[i] = valve_patient_list[i].split()

print(valve_patient_list)

['[강아지 심장 판막 질환 환자]', ['1038', '장수'], ['1087', '울'], ['1168', '별님'], ['1269', '셜록'], ['1329', '짱이'], ['1369', '뭉치'], ['1403', '터치'], ['1407', '안나'], ['1409', '정유리'], ['143', '만두'], ['1474', '망고'], ['1617', '짱구'], ['1684', '니니'], ['1713', '보리'], ['1770', '밀크'], ['1844', '동동이'], ['1845', '김아리'], ['1869', '나나'], ['1923', '하루'], ['1938', '잔비'], ['1958', '구름'], ['2037', '구찌'], ['2056', '써니'], ['210', '몰랑이'], ['229', '지니'], ['2317', '곰'], ['2367', '별이'], ['2482', '꽁지'], ['2554', '초롱이'], ['2557', '소미'], ['2559', '보리'], ['2574', '또또'], ['2583', '꽃순이'], ['2669', '하임'], ['2694', '봄'], ['2710', '앵두'], ['273', '보리'], ['283', '오월이'], ['2882', '탄이'], ['291', '찹쌀'], ['2933', '강한'], ['30', '로씨', 'Roxy'], ['3073', '황군'], ['3099', '미니'], ['31', '기즈모', 'Gizmo'], ['3138', '몽실이'], ['3146', '예삐'], ['3216', '투투'], ['3277', '방울'], ['3388', '모모'], ['3390', '코코'], ['3472', '와꾸'], ['3539', '수리'], ['3540', '오공이'], ['3572', '소미'], ['3601', '초롱'], ['4', '예삐'], ['406', '막둥이'], ['408', '겨울'], ['493', '아지'], ['565', '

In [17]:
valve_list = []

for i in range(1, len(valve_patient_list)-1):
    if len(valve_patient_list[i]) == 0:
        print("강아지 심장 판막 질환 환자")
        break
    valve_list.append(int(valve_patient_list[i][0]))
    

print(valve_list)

강아지 심장 판막 질환 환자
[1038, 1087, 1168, 1269, 1329, 1369, 1403, 1407, 1409, 143, 1474, 1617, 1684, 1713, 1770, 1844, 1845, 1869, 1923, 1938, 1958, 2037, 2056, 210, 229, 2317, 2367, 2482, 2554, 2557, 2559, 2574, 2583, 2669, 2694, 2710, 273, 283, 2882, 291, 2933, 30, 3073, 3099, 31, 3138, 3146, 3216, 3277, 3388, 3390, 3472, 3539, 3540, 3572, 3601, 4, 406, 408, 493, 565, 567, 568, 648, 651, 672, 677, 678, 689, 713, 730, 802, 89, 903, 94]


In [18]:
len(valve_list)

75

In [19]:
ah1_df['Label'] = 0

ah1_df.loc[ah1_df['Pet.id'].isin(valve_list), 'Label'] = 1 # 심장병환자

In [20]:
ah1_df["Label"].value_counts()

Label
0    21785
1     1477
Name: count, dtype: int64

In [21]:
ah1_df = ah1_df.drop("PetDiagnosisImage", axis=1)

ah1_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label
0,NaN,2022-02-07T10:27:01.920Z,[],[],[],[],1,알수없음,UNKNOWN,0.00,NaN,0
1,애드보킷 구매,2022-02-07T10:26:23.647Z,[],[],[],['BE4.[Pack]애드보킷-CAT 4-8kg'],297,신구름,CM,5.30,NaN,0
2,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.91,<p>S</p>\n<p>건강했음.</p>\n<p>어제 침을 너무 많이 흘린 것 이외...,0
3,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.60,<p>S</p>\n<p>- 특이사항 없었음</p>\n<p>O</p>\n<p>- 오른...,0
4,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.38,<p>S</p>\n<p>더 나아지지는 않는 것 같다고 말씀</p>\n<p>활력은 양...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23257,NaN,2024-05-17T03:20:39.751Z,[],[],"['[수액] N/S, H/S 수액 처치_100ml', '[치과] 치석제거, 스켈링 ...",[],568,상추,CM,4.35,NaN,1
23258,재진 구토,2024-05-17T02:05:34.182Z,"['Metoclopramide, 메토클로프라마이드', 'Metronidazole, ...",['포티플로라 1파우치'],"['특수주사_세레니아(항구토제)_10kg 이하', '방사선 복부 2컷 10kg 미만...",[],2045,몽실,SF,5.50,<p>S</p><p>노란색 거품토를 어제 4번 하고 오늘 아침에 새벽 6시에 한번 ...,0
23259,상담,2024-05-17T02:01:27.360Z,[],[],['[진료비] 상담비(동물 미동반)'],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",2052,루,CM,6.30,<p>미동반으로 건강검진 상담위해 내원하심.</p><p>: standard + 췌장...,0
23260,NaN,2024-05-17T01:51:48.524Z,[],[],['힐스 독 z/d 1.5kg'],[],1739,탄,CM,7.70,NaN,0


In [22]:
ah1_df['chiefComplaint'] = ah1_df['chiefComplaint'].fillna("")
ah1_df = ah1_df.dropna(axis = 'index', how = 'any', subset = ['Subjective.visibleText'])

In [23]:
ah1_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label
2,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.910,<p>S</p>\n<p>건강했음.</p>\n<p>어제 침을 너무 많이 흘린 것 이외...,0
3,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.600,<p>S</p>\n<p>- 특이사항 없었음</p>\n<p>O</p>\n<p>- 오른...,0
4,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.380,<p>S</p>\n<p>더 나아지지는 않는 것 같다고 말씀</p>\n<p>활력은 양...,0
5,,2022-02-07T08:12:32.807Z,[],[],[],[],1347,아롱이,SF,3.380,<p></p>,0
7,심장사상충,2022-02-07T07:48:12.477Z,[],[],[],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",1907,하루,SF,4.595,<p>S</p>\n<p>심장사상충 예방하기 위해 내원</p>\n<p>최근에 사료토<...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23254,술부확인,2024-05-17T05:17:44.477Z,[],[],[],[],3304,용검이,M,2.850,<p>술부 잘 아물었음</p><p>2 sutures 제거 완료 </p><p>박 </p>,0
23255,재진 혈토,2024-05-17T05:05:19.106Z,"['Famotidine, 파모티딘', 'Metoclopramide, 메토클로프라마이...",[],"['[예방] 구충제_안텔민 뽀삐 1T', '[처방] 내복약 1일 2회_5kg 이하'...",[],2833,보리,CM,3.995,<p>S</p><p>어머니가 혈토를 한군데 발견했음</p><p>연한 붉은색이 섞여있...,0
23258,재진 구토,2024-05-17T02:05:34.182Z,"['Metoclopramide, 메토클로프라마이드', 'Metronidazole, ...",['포티플로라 1파우치'],"['특수주사_세레니아(항구토제)_10kg 이하', '방사선 복부 2컷 10kg 미만...",[],2045,몽실,SF,5.500,<p>S</p><p>노란색 거품토를 어제 4번 하고 오늘 아침에 새벽 6시에 한번 ...,0
23259,상담,2024-05-17T02:01:27.360Z,[],[],['[진료비] 상담비(동물 미동반)'],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",2052,루,CM,6.300,<p>미동반으로 건강검진 상담위해 내원하심.</p><p>: standard + 췌장...,0


In [62]:
ah1_blood = pd.read_csv(file_path + "ah1/Test.csv")

ah1_blood = ah1_blood.drop(['completedDateTime', 'unit', 'value', 'minRefer', 'maxRefer', 'testName', 'instrumentName'], axis=1)
ah1_blood = ah1_blood.drop_duplicates()
ah1_blood = ah1_blood.reset_index(drop=True)

li = []
for i in range(len(ah1_blood)):
    li.append('혈액검사')
df = pd.DataFrame(columns=['TestName'], data=li)

ah1_blood = pd.concat([ah1_blood, df], axis=1)

ah1_blood

,orderedDateTime,Pet.id,Pet.petName,TestName
0,2021-09-07T01:07:47.047Z,307,가을,혈액검사
1,2021-09-05T01:43:02.370Z,1326,콜라,혈액검사
2,2021-09-05T03:18:50.690Z,1053,홍시,혈액검사
3,2021-09-05T02:02:24.793Z,3,팡이,혈액검사
4,2021-09-04T01:11:31.363Z,1340,감자,혈액검사
...,...,...,...,...
2618,2024-05-15T02:16:46.797Z,2671,두부,혈액검사
2619,2024-05-15T03:18:50.139Z,3510,이누리,혈액검사
2620,2024-05-15T02:16:47.103Z,2671,두부,혈액검사
2621,2024-05-15T03:18:50.419Z,3510,이누리,혈액검사


In [64]:
ah1_blood.rename(columns={'orderedDateTime':'diagnosisDateTime'}, inplace=True)

ah1_blood

,diagnosisDateTime,Pet.id,Pet.petName,TestName
0,2021-09-07T01:07:47.047Z,307,가을,혈액검사
1,2021-09-05T01:43:02.370Z,1326,콜라,혈액검사
2,2021-09-05T03:18:50.690Z,1053,홍시,혈액검사
3,2021-09-05T02:02:24.793Z,3,팡이,혈액검사
4,2021-09-04T01:11:31.363Z,1340,감자,혈액검사
...,...,...,...,...
2618,2024-05-15T02:16:46.797Z,2671,두부,혈액검사
2619,2024-05-15T03:18:50.139Z,3510,이누리,혈액검사
2620,2024-05-15T02:16:47.103Z,2671,두부,혈액검사
2621,2024-05-15T03:18:50.419Z,3510,이누리,혈액검사


In [65]:
ah1_df = pd.merge(ah1_df, ah1_blood, how='left', on=['diagnosisDateTime', 'Pet.id', 'Pet.petName'])

ah1_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label,TestName
0,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.910,<p>S</p>\n<p>건강했음.</p>\n<p>어제 침을 너무 많이 흘린 것 이외...,0,NaN
1,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.600,<p>S</p>\n<p>- 특이사항 없었음</p>\n<p>O</p>\n<p>- 오른...,0,NaN
2,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.380,<p>S</p>\n<p>더 나아지지는 않는 것 같다고 말씀</p>\n<p>활력은 양...,0,NaN
3,,2022-02-07T08:12:32.807Z,[],[],[],[],1347,아롱이,SF,3.380,<p></p>,0,NaN
4,심장사상충,2022-02-07T07:48:12.477Z,[],[],[],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",1907,하루,SF,4.595,<p>S</p>\n<p>심장사상충 예방하기 위해 내원</p>\n<p>최근에 사료토<...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15702,술부확인,2024-05-17T05:17:44.477Z,[],[],[],[],3304,용검이,M,2.850,<p>술부 잘 아물었음</p><p>2 sutures 제거 완료 </p><p>박 </p>,0,NaN
15703,재진 혈토,2024-05-17T05:05:19.106Z,"['Famotidine, 파모티딘', 'Metoclopramide, 메토클로프라마이...",[],"['[예방] 구충제_안텔민 뽀삐 1T', '[처방] 내복약 1일 2회_5kg 이하'...",[],2833,보리,CM,3.995,<p>S</p><p>어머니가 혈토를 한군데 발견했음</p><p>연한 붉은색이 섞여있...,0,NaN
15704,재진 구토,2024-05-17T02:05:34.182Z,"['Metoclopramide, 메토클로프라마이드', 'Metronidazole, ...",['포티플로라 1파우치'],"['특수주사_세레니아(항구토제)_10kg 이하', '방사선 복부 2컷 10kg 미만...",[],2045,몽실,SF,5.500,<p>S</p><p>노란색 거품토를 어제 4번 하고 오늘 아침에 새벽 6시에 한번 ...,0,NaN
15705,상담,2024-05-17T02:01:27.360Z,[],[],['[진료비] 상담비(동물 미동반)'],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",2052,루,CM,6.300,<p>미동반으로 건강검진 상담위해 내원하심.</p><p>: standard + 췌장...,0,NaN


In [24]:
# 2. ah2 (time)

ah2_chart = pd.read_csv(file_path + "ah2/chart.csv", encoding='utf-8')
ah2_interpretation = pd.read_csv(file_path + "ah2/interpretation.csv", encoding='cp949') # 진단 내용
ah2_pet = pd.read_csv(file_path + "ah2/pet.csv", encoding='utf-8') # pet 정보
ah2_breed = pd.read_csv(file_path + "ah2/breed_name_label.csv", encoding='utf-8') # 품종 정보: 강아지 분류
ah2_order2016 = pd.read_csv(file_path + "ah2/처방_2016.csv", encoding='utf-8')
ah2_order2017 = pd.read_csv(file_path + "ah2/처방_2017.csv", encoding='utf-8')
ah2_order2018 = pd.read_csv(file_path + "ah2/처방_2018.csv", encoding='utf-8')
ah2_order2019 = pd.read_csv(file_path + "ah2/처방_2019.csv", encoding='utf-8')
ah2_order2020 = pd.read_csv(file_path + "ah2/처방_2020.csv", encoding='utf-8')
ah2_order2021 = pd.read_csv(file_path + "ah2/처방_2021.csv", encoding='utf-8')
ah2_order2022 = pd.read_csv(file_path + "ah2/처방_2022.csv", encoding='utf-8')
ah2_order2023 = pd.read_csv(file_path + "ah2/처방_2023.csv", encoding='utf-8')
ah2_order2024 = pd.read_csv(file_path + "ah2/처방_2024.csv", encoding='utf-8')

In [25]:
ah2_breed

,breed_name,label,Unnamed: 2
0,Norwegian Forest,c,NaN
1,Pomeranian,d,NaN
2,Poodle,d,NaN
3,Maltese,d,NaN
4,Korat,c,NaN
...,...,...,...
590,?? ??,d,NaN
591,Persian mix,c,NaN
592,Pomeranian+Poodle,d,NaN
593,Labrador Retriever Mix,d,NaN


In [26]:
ah2_breed = ah2_breed.drop("Unnamed: 2", axis=1)

ah2_breed

,breed_name,label
0,Norwegian Forest,c
1,Pomeranian,d
2,Poodle,d
3,Maltese,d
4,Korat,c
...,...,...
590,?? ??,d
591,Persian mix,c
592,Pomeranian+Poodle,d
593,Labrador Retriever Mix,d


In [27]:
ah2_breed[ah2_breed['label'] == '?'] = 'd'

In [28]:
ah2_breed[ah2_breed['label'] == 'd']

,breed_name,label
1,Pomeranian,d
2,Poodle,d
3,Maltese,d
6,Pekingese,d
7,d,d
...,...,...
589,Beagle Mix,d
590,?? ??,d
592,Pomeranian+Poodle,d
593,Labrador Retriever Mix,d


In [29]:
ah2_pet

,pet_serial,pet_name,breed_name,pet_color,pet_birth,sex_id
0,26,치치,Norwegian Forest,Black and White,2016-06-08,Female
1,27,보리,Pomeranian,White,2016-02-08,중성화 Male
2,29,댕이,Poodle,Apricot,2013-10-01,Female
3,30,율무,Maltese,White,2014-03-01,Female
4,31,델피,Korat,NaN,2010-08-09,중성화 Female
...,...,...,...,...,...,...
32691,32887,초롱이,Poodle,NaN,2016-08-06,Female
32692,32888,봉달이,Bichon Frise,NaN,2018-08-06,중성화 Male
32693,32889,곱슬이,Bichon Frise,NaN,2019-08-06,중성화 Male
32694,32890,시루,Ragdoll,NaN,2024-02-06,Female


In [30]:
ah2_pet_dog = pd.merge(ah2_pet, ah2_breed, on='breed_name', how='left')

ah2_pet_dog

,pet_serial,pet_name,breed_name,pet_color,pet_birth,sex_id,label
0,26,치치,Norwegian Forest,Black and White,2016-06-08,Female,c
1,27,보리,Pomeranian,White,2016-02-08,중성화 Male,d
2,29,댕이,Poodle,Apricot,2013-10-01,Female,d
3,30,율무,Maltese,White,2014-03-01,Female,d
4,31,델피,Korat,NaN,2010-08-09,중성화 Female,c
...,...,...,...,...,...,...,...
32691,32887,초롱이,Poodle,NaN,2016-08-06,Female,d
32692,32888,봉달이,Bichon Frise,NaN,2018-08-06,중성화 Male,d
32693,32889,곱슬이,Bichon Frise,NaN,2019-08-06,중성화 Male,d
32694,32890,시루,Ragdoll,NaN,2024-02-06,Female,c


In [31]:
ah2_pet_dog = ah2_pet_dog[ah2_pet_dog['label'] == 'd']

ah2_pet_dog

,pet_serial,pet_name,breed_name,pet_color,pet_birth,sex_id,label
1,27,보리,Pomeranian,White,2016-02-08,중성화 Male,d
2,29,댕이,Poodle,Apricot,2013-10-01,Female,d
3,30,율무,Maltese,White,2014-03-01,Female,d
5,32,바비,Poodle,Tan and Tan,2009-08-09,중성화 Female,d
7,34,복길이,Pekingese,White,2003-08-09,Female,d
...,...,...,...,...,...,...,...
32689,32885,순둥이,Pomeranian,Buff,2019-08-06,중성화 Male,d
32690,32886,강이,Shih Tzu,NaN,2008-08-06,중성화 Male,d
32691,32887,초롱이,Poodle,NaN,2016-08-06,Female,d
32692,32888,봉달이,Bichon Frise,NaN,2018-08-06,중성화 Male,d


In [32]:
dog_id = ah2_pet_dog['pet_serial'][ah2_pet_dog['label'] == 'd']

dog_id

1           27
2           29
3           30
5           32
7           34
         ...  
32689    32885
32690    32886
32691    32887
32692    32888
32693    32889
Name: pet_serial, Length: 21698, dtype: int64

In [33]:
ah2_chart_dog = ah2_chart[ah2_chart['pet_serial'].isin(dog_id)]

ah2_chart_dog

,pet_serial,pet_name,chart_attending,chart_date,chart_cc,chartdesc_title,chartdesc_txtmemo
2,291,미니,윤문수,2016-04-11,NaN,NaN,NaN
3,418,육월이,윤문수,2016-04-18,NaN,NaN,NaN
5,598,봄,이남순,2016-06-01,NaN,NaN,NaN
6,579,쿠키,이남순,2016-06-01,NaN,NaN,NaN
7,616,슈슈,이남순,2016-06-02,NaN,NaN,NaN
...,...,...,...,...,...,...,...
275258,16657,루비,노영은,2024-07-31,"백신부작용 후 장염, 췌장염가능성",NaN,S)\r\n- 2-3일 멎었다가 다시 시작\r\n- 그제 설사 1-2번\r\n- 어...
275262,32822,잼,한지희,2024-07-31,간식 먹고 켁켁거림,NaN,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에..."
275264,24202,앵두,이철호,2024-07-31,초콜릿 섭취(1시간 반전)//야고//3층대고,NaN,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...
275265,30604,베니,이철호,2024-07-31,호흡 가쁨//야고//3층대고,NaN,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...


In [34]:
ah2_chart_dog = ah2_chart_dog.reset_index(drop=True)

ah2_chart_dog

,pet_serial,pet_name,chart_attending,chart_date,chart_cc,chartdesc_title,chartdesc_txtmemo
0,291,미니,윤문수,2016-04-11,NaN,NaN,NaN
1,418,육월이,윤문수,2016-04-18,NaN,NaN,NaN
2,598,봄,이남순,2016-06-01,NaN,NaN,NaN
3,579,쿠키,이남순,2016-06-01,NaN,NaN,NaN
4,616,슈슈,이남순,2016-06-02,NaN,NaN,NaN
...,...,...,...,...,...,...,...
201663,16657,루비,노영은,2024-07-31,"백신부작용 후 장염, 췌장염가능성",NaN,S)\r\n- 2-3일 멎었다가 다시 시작\r\n- 그제 설사 1-2번\r\n- 어...
201664,32822,잼,한지희,2024-07-31,간식 먹고 켁켁거림,NaN,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에..."
201665,24202,앵두,이철호,2024-07-31,초콜릿 섭취(1시간 반전)//야고//3층대고,NaN,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...
201666,30604,베니,이철호,2024-07-31,호흡 가쁨//야고//3층대고,NaN,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...


In [35]:
order_df = pd.concat([ah2_order2016, ah2_order2017, ah2_order2018, ah2_order2019, ah2_order2020, ah2_order2021, ah2_order2022, ah2_order2023, ah2_order2024])

order_df

,pet_serial,pet_name,chart_date,plan_name,created_sign_name
0,27,보리,2016-08-08,진료비-기본1,김재훈
1,27,보리,2016-08-08,X-ray (기본1cut),김재훈
2,27,보리,2016-08-08,내복약-1일 2회(1-5kg),김재훈
3,26,치치,2016-08-08,진료비-초진4,관리자
4,26,치치,2016-08-08,혈액(혈구)-CBC,관리자
...,...,...,...,...,...
133063,32823,코코,2024-07-31,혈액(혈구)-CBC(procyte) (2024),이철호
133064,32823,코코,2024-07-31,"혈액(가스)-gas 가스분석, 전해질검사(ABL) (2024)",이철호
133065,24916,마이콜,2024-07-26,귀세정제 - Tris EDTA 120ml 1EA,권선우
133066,32776,김메코,2024-07-31,"외과 기본입원 비용 5kg 이하 -일반외과, 정형 등",김지수


In [36]:
order_df = order_df.dropna(axis = 'index', how = 'any', subset = ['plan_name'])

In [37]:
order_df = order_df.reset_index(drop=True)

In [38]:
diabetes_list = []

for i in range(len(order_df)):
    # print(i)
    if "인슐린" in order_df['plan_name'][i]:
        diabetes_list.append(order_df['pet_serial'][i])

diabetes_list

[554,
 1053,
 554,
 1866,
 1866,
 1866,
 1609,
 1866,
 2997,
 1866,
 554,
 1866,
 4344,
 1866,
 554,
 554,
 1866,
 5372,
 5372,
 5531,
 5372,
 6111,
 554,
 1866,
 7717,
 8325,
 7763,
 1866,
 8758,
 7763,
 7681,
 7681,
 9578,
 7681,
 7717,
 10024,
 10024,
 554,
 6504,
 8758,
 10360,
 7763,
 1866,
 6504,
 1866,
 6504,
 10600,
 8325,
 8758,
 10600,
 10600,
 10999,
 11002,
 6504,
 11067,
 10600,
 6504,
 6504,
 7763,
 7717,
 11535,
 11535,
 6504,
 8758,
 11578,
 11578,
 6504,
 11578,
 6205,
 6205,
 6504,
 10933,
 6205,
 12027,
 12027,
 12027,
 6504,
 9578,
 1866,
 11578,
 12141,
 7763,
 6504,
 12027,
 554,
 10600,
 8758,
 6205,
 12027,
 12615,
 12615,
 6504,
 12479,
 10308,
 12615,
 10600,
 6504,
 10308,
 12927,
 8758,
 6205,
 6504,
 13312,
 13312,
 13020,
 13312,
 13312,
 11578,
 12615,
 13312,
 1866,
 10308,
 6205,
 6504,
 7717,
 13312,
 8325,
 13020,
 10600,
 9578,
 6504,
 10933,
 554,
 6504,
 11578,
 13020,
 12615,
 11067,
 6504,
 6504,
 10600,
 1866,
 6504,
 10308,
 6504,
 9393,
 9578,

In [39]:
valve_list = []

for i in range(len(order_df)):
    # print(i)
    if "심장약" in order_df['plan_name'][i]:
        valve_list.append(order_df['pet_serial'][i])

valve_list

[124,
 163,
 163,
 163,
 124,
 163,
 163,
 124,
 124,
 124,
 163,
 163,
 124,
 355,
 579,
 579,
 163,
 163,
 124,
 579,
 579,
 684,
 163,
 163,
 754,
 754,
 579,
 579,
 754,
 754,
 163,
 163,
 579,
 579,
 124,
 754,
 754,
 579,
 579,
 163,
 163,
 124,
 1005,
 1005,
 579,
 579,
 754,
 754,
 1005,
 1005,
 989,
 989,
 124,
 1005,
 1005,
 1126,
 1005,
 1005,
 1126,
 1241,
 1241,
 1005,
 1005,
 1304,
 731,
 731,
 124,
 1005,
 1005,
 802,
 802,
 1304,
 1376,
 1376,
 754,
 754,
 1376,
 1376,
 1421,
 1421,
 1005,
 1005,
 989,
 989,
 754,
 754,
 731,
 731,
 1376,
 1376,
 1484,
 1484,
 1493,
 1509,
 731,
 731,
 124,
 163,
 163,
 1005,
 1005,
 731,
 731,
 163,
 1509,
 1304,
 163,
 1609,
 731,
 731,
 1609,
 1509,
 163,
 124,
 1005,
 1005,
 163,
 163,
 1509,
 1609,
 731,
 731,
 1304,
 163,
 1005,
 1005,
 124,
 163,
 163,
 1509,
 1609,
 1304,
 163,
 163,
 1948,
 1948,
 1995,
 1893,
 1948,
 1948,
 163,
 163,
 1995,
 1005,
 1005,
 1509,
 1609,
 1948,
 1948,
 1893,
 124,
 1995,
 163,
 163,
 731,
 731,


In [40]:
valve_list = list(set(valve_list))

len(valve_list)

1281

In [41]:
diabetes_list = list(set(diabetes_list))

len(diabetes_list)

133

In [42]:
ah2_df = ah2_chart.drop("chart_attending", axis=1)
ah2_df['Label'] = 0

ah2_df.loc[ah2_df['pet_serial'].isin(valve_list), 'Label'] = 1 # 심장병환자
ah2_df.loc[ah2_df['pet_serial'].isin(diabetes_list), 'Label'] = 2 # 당뇨병환자

In [43]:
ah2_df

,pet_serial,pet_name,chart_date,chart_cc,chartdesc_title,chartdesc_txtmemo,Label
0,2509,초롱이,2017-03-20,3/19 입원 모니터,3/19 입원 모니터,[3/19 입원 모니터 by 서지민]\r\n- 12시경 케이지내에서 jumping\...,0
1,2905,초초,2017-04-18,4/17 입원 모니터,4/17 입원 모니터,[4/17 입원 모니터 by 서지민]\r\n- T 37.0\r\n- 수액처치 후 2...,0
2,291,미니,2016-04-11,NaN,NaN,NaN,0
3,418,육월이,2016-04-18,NaN,NaN,NaN,0
4,577,공주,2016-06-01,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...
275262,32822,잼,2024-07-31,간식 먹고 켁켁거림,NaN,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에...",0
275263,10563,여름이,2024-07-31,구강출혈//야고//케이지 무게만,NaN,S)\r\n발바닥에 피가 묻어있길래 패드가 어디에 찔렸나 싶었는데\r\n입이 자꾸 ...,0
275264,24202,앵두,2024-07-31,초콜릿 섭취(1시간 반전)//야고//3층대고,NaN,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...,0
275265,30604,베니,2024-07-31,호흡 가쁨//야고//3층대고,NaN,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...,0


In [44]:
ah2_df['Label'].value_counts()

Label
0    228863
1     40022
2      6382
Name: count, dtype: int64

In [46]:
ah2_df_dog = ah2_df[ah2_df['pet_serial'].isin(dog_id)]

ah2_df_dog

,pet_serial,pet_name,chart_date,chart_cc,chartdesc_title,chartdesc_txtmemo,Label
2,291,미니,2016-04-11,NaN,NaN,NaN,0
3,418,육월이,2016-04-18,NaN,NaN,NaN,0
5,598,봄,2016-06-01,NaN,NaN,NaN,0
6,579,쿠키,2016-06-01,NaN,NaN,NaN,1
7,616,슈슈,2016-06-02,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...
275258,16657,루비,2024-07-31,"백신부작용 후 장염, 췌장염가능성",NaN,S)\r\n- 2-3일 멎었다가 다시 시작\r\n- 그제 설사 1-2번\r\n- 어...,0
275262,32822,잼,2024-07-31,간식 먹고 켁켁거림,NaN,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에...",0
275264,24202,앵두,2024-07-31,초콜릿 섭취(1시간 반전)//야고//3층대고,NaN,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...,0
275265,30604,베니,2024-07-31,호흡 가쁨//야고//3층대고,NaN,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...,0


In [47]:
ah2_df = ah2_df_dog.reset_index(drop=True)

In [51]:
re_col = ['Pet.id', 'Pet.petName', 'diagnosisDateTime', 'chiefComplaint', 'chartdesc_title', 'Subjective.visibleText', 'Label']

ah2_df.columns = re_col

ah2_df

,Pet.id,Pet.petName,diagnosisDateTime,chiefComplaint,chartdesc_title,Subjective.visibleText,Label
0,291,미니,2016-04-11,NaN,NaN,NaN,0
1,418,육월이,2016-04-18,NaN,NaN,NaN,0
2,598,봄,2016-06-01,NaN,NaN,NaN,0
3,579,쿠키,2016-06-01,NaN,NaN,NaN,1
4,616,슈슈,2016-06-02,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...
201663,16657,루비,2024-07-31,"백신부작용 후 장염, 췌장염가능성",NaN,S)\r\n- 2-3일 멎었다가 다시 시작\r\n- 그제 설사 1-2번\r\n- 어...,0
201664,32822,잼,2024-07-31,간식 먹고 켁켁거림,NaN,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에...",0
201665,24202,앵두,2024-07-31,초콜릿 섭취(1시간 반전)//야고//3층대고,NaN,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...,0
201666,30604,베니,2024-07-31,호흡 가쁨//야고//3층대고,NaN,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...,0


In [52]:
ah2_df['chiefComplaint'] = ah2_df['chiefComplaint'].fillna("")
ah2_df['chartdesc_title'] = ah2_df['chartdesc_title'].fillna("")

ah2_df = ah2_df.dropna(axis = 'index', how = 'any', subset = ['Subjective.visibleText'])

ah2_df

,Pet.id,Pet.petName,diagnosisDateTime,chiefComplaint,chartdesc_title,Subjective.visibleText,Label
13,27,보리,2016-08-08,꼬리를 만지면 아파합니다,꼬리를 만지면 아파합니다,S)\r\n- 주말부터 꼬리를 만지면 아파합니다\r\n- Wax and wane으로...,0
14,30,율무,2016-08-09,초콜렛 섭취,초콜렛 섭취,\r C.C) 초코렛 섭취\r\r S) 초코렛 섭취 \r 새벽 2시에서 5시...,0
15,29,댕이,2016-08-09,초콜렛섭취,초콜렛섭취,C.C) 초코렛 섭취\r\n\r\n S) 초코렛 섭취 \r\n 새벽 2시에서...,0
17,34,복길이,2016-08-09,식욕저하,식욕저하,H)\r\n4일 전부터 식욕저하\r\n변묽음\r\n\r\n10년전 디스크의증으로 체...,0
19,29,댕이,2016-08-09,슬개골탈구 상담,슬개골탈구 상담,다음주에 엑스레이찍어보고 수술결정하시면 혈액검사비용 빼드리기로함\r\n댕이 양측 3...,0
...,...,...,...,...,...,...,...
201663,16657,루비,2024-07-31,"백신부작용 후 장염, 췌장염가능성",,S)\r\n- 2-3일 멎었다가 다시 시작\r\n- 그제 설사 1-2번\r\n- 어...,0
201664,32822,잼,2024-07-31,간식 먹고 켁켁거림,,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에...",0
201665,24202,앵두,2024-07-31,초콜릿 섭취(1시간 반전)//야고//3층대고,,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...,0
201666,30604,베니,2024-07-31,호흡 가쁨//야고//3층대고,,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...,0


In [53]:
ah2_df['Label'].value_counts()

Label
0    145763
1     30698
2      4509
Name: count, dtype: int64

In [56]:
# ah2_df[(ah2_df['Pet.petName'] == '루카') | (ah2_df['Label'] == 1)]

ah2_df[(ah2_df['Subjective.visibleText'].map(len) < 25)] #  & (ah2_df['Label'] == 1)

,Pet.id,Pet.petName,diagnosisDateTime,chiefComplaint,chartdesc_title,Subjective.visibleText,Label
86,114,뭉치,2016-08-19,,,●,0
117,145,꼬꼬미,2016-08-21,심장사상충/기본정리,심장사상충/기본정리,우측 슬개골탈구 1기소견,0
125,104,미유,2016-08-22,예방접종,예방접종,8/22\r\n\r\n 일반상태 양호. \r\n,0
174,7,김초코,2016-08-28,"입원중모니터링 (아침, 저녁 처치 체크)","입원중모니터링 (아침, 저녁 처치 체크)",집중관리,0
206,208,고봉,2016-08-29,,,경구약\r\nAMC\r\nPredniso,0
...,...,...,...,...,...,...,...
201498,32796,몽이,2024-07-30,7월 29일 야간,,식이 1회 거부\r\n배뇨 1회\r\n과흥분 지속,0
201554,29407,우즈,2024-07-30,"로얄캐닌 lowfat, 힐스 lowfat 추천",,"I/D 로우펫 건*1 , 캔*2 구매",0
201556,4413,콩이,2024-07-30,DTM 배양,DTM 배양,DTM 배양 1주차 경과 -> (-)?,0
201646,32524,정진달래,2024-07-31,"CT 1차소견, 프로포폴14 케타민 10ml 사용","CT 1차소견, 프로포폴 사용",프로포폴 14ml 사용-유지민,1


In [71]:
ah2_blood

,pet_serial,pet_name,labdate_date
0,42,심슨,2016-08-10
1,42,심슨,2016-08-10
2,42,심슨,2016-08-10
3,42,심슨,2016-08-10
4,42,심슨,2016-08-10
...,...,...,...
1637336,32823,코코,2024-07-31
1637337,32823,코코,2024-07-31
1637338,32823,코코,2024-07-31
1637339,32823,코코,2024-07-31


In [72]:
# ah2_blood = ah2_blood.drop(['labdate_title', 'labitem_name', 'labresult_value'], axis=1)
ah2_blood = ah2_blood.drop_duplicates()
ah2_blood = ah2_blood.reset_index(drop=True)

li = []
for i in range(len(ah2_blood)):
    li.append('혈액검사')
df = pd.DataFrame(columns=['TestName'], data=li)

ah2_blood = pd.concat([ah2_blood, df], axis=1)
ah2_blood = ah2_blood.rename(columns={'labdate_date':'chart_date'})

ah2_blood

,pet_serial,pet_name,chart_date,TestName
0,42,심슨,2016-08-10,혈액검사
1,45,돈이,2016-08-11,혈액검사
2,48,부루,2016-08-11,혈액검사
3,55,옹이,2016-08-12,혈액검사
4,59,콩지,2016-08-12,혈액검사
...,...,...,...,...
53767,32808,겨울,2024-07-31,혈액검사
53768,18576,아톰,2024-07-31,혈액검사
53769,32821,길냥이,2024-07-31,혈액검사
53770,16657,루비,2024-07-31,혈액검사


In [85]:
ah2_test = pd.read_csv(file_path + "ah2/interpretation.csv", encoding='cp949')

for i in range(len(ah2_test)):
    if type(ah2_test.loc[i, 'study_interpretation']) != float:
        ah2_test.loc[i, 'study_interpretation'] = ah2_test.loc[i, 'study_interpretation'].replace("[", "")
        ah2_test.loc[i, 'study_interpretation'] = ah2_test.loc[i, 'study_interpretation'].replace(")\r\n", "\r\n")
        ah2_test.loc[i, 'study_interpretation'] = ah2_test.loc[i, 'study_interpretation'].replace("]", "\r\n")

ah2_test = ah2_test.dropna(axis = 'index', how = 'any', subset = ['study_interpretation'])

ah2_test

,pet_serial,pet_name,study_date,study_desc,study_interpretation
0,194,사랑,2016-09-10 0:00,THORAX VD/LAT,후지 방사선\r\n 후지 방사선\r\n-양측 슬개골 내측 탈구. 두드러진 관절낭 부...
1,361,모리,2016-09-11 0:00,THORAX VD/LAT,흉부 방사선\r\n 흉부 방사선\r\n-심종대 (약 4늑간\r\n-우심종대. 주폐동...
2,466,가지,2016-09-16 0:00,THORAX VD/LAT,후지 방사선\r\n 후지 방사선\r\n-슬개골 위치정상\r\n-슬관절에 두드러진 퇴...
3,544,꾹이,2016-09-23 0:00,NaN,흉부 방사선\r\n 흉부 방사선\r\n-심장 크기 및 모양 정상\r\n-양측 후엽과...
4,42,심슨,2016-09-19 0:00,THORAX LAT,흉부 방사선\r\n -심장 크기 정상. 복배상 경미한 우심방 확장 소견\r\n-폐실...
...,...,...,...,...,...
72974,16657,루비,2024-07-31 21:18,\N,"소화기, 췌장\r\n- 결장 내 연변양상의 분변 관찰\r\n- 결장 벽 두께 정상범..."
72975,16657,루비,2024-07-31 21:12,\N,복부방사선\r\n- 선예도 양호\r\n- 복강 장기 특이소견 관찰되지 않음\r\n\...
72976,32822,잼,2024-07-31 22:21,\N,"흉부 방사선 \r\n - 식도 조영 상, 후방 식도~위 연접부의 이물(8*1.2cm..."
72977,24202,앵두,2024-07-31 23:22,\N,"복부 방사선 \r\n- 위 내용물 다량 확인되며, 구토 처치 후 대부분 배출됨\r\..."


In [163]:
test = ['술부, 비뇨기 초음파',
 '후지 술후 방사선',
 '간 담도 초음파',
 '비장 절제 부위 초음파',
 '소화기+대동맥 초음파',
 '비뇨기. 회음부 초음파',
 '간담도, 비뇨기 초음파',
 '방광천자',
 '간이 흉강 초음파',
 '방광결석 제거술후 방사선',
 '꼬리 방사선',
 '우심 초음파',
 '소화기, 췌장 초음파',
 '인후두 방사선',
 '복부간이초음파',
 '술후 간담도 초음파',
 '소화기, 신장 초음파',
 '피하 병변 초음파',
 '항문 주위 초음파',
 '결절 초음파',
 '간, 위 초음파',
 'SUB 제거 후 비뇨기 초음파',
 '주폐동맥 초음파',
 '두개골 인후두 방사선',
 '술부 국소 초음파',
 '단일 장기 초음파',
 '침샘 초음파',
 '지역병원 방사선 평가',
 '요도방사선',
 '술후 소화기 초음파',
 '간이 소화기 초음파',
 '좌측 견관절 술후 방사선',
 '간담도',
 '흉부, 인후두부 방사선',
 '간, 복수 초음파',
 'MR 최종',
 '비뇨기 생식기 초음파',
 '전지 발가락 방사선',
 '방광 술후 초음파',
 '우측 후지 초음파',
 '술부 및 소화기 초음파',
 '횡격막 파열 술 후 방사선',
 '우측 후지 발가락 술후 방사선',
 '좌측 전지 발가락 골절',
 '하악아래 림프절 주변 초음파',
 '뇨관결석 초음파',
 '비뇨 생식기, 부신 초음파',
 'TL MR',
 '좌측 요관 결석',
 '잠복고환 초음파',
 '전지 관절 초음파',
 '요추 방사선',
 '흉수 약 110ml 천자',
 '자궁초음파',
 'MRI 차',
 '좌측 후지 초음파',
 '요척골 골절 술후 방사선',
 'Abdomen CT',
 '간이 폐초음파',
 '장초음파',
 '흉골과 좌측 상완골 주변 초음파',
 '복수 확인, 항문낭 초음파',
 '견갑 방사선',
 '담낭초음파',
 '간, 비장과 비뇨기 초음파',
 '흉수 약 35cc천자',
 '생식기 단일 초음파',
 '담도.췌장 초음파',
 '담낭, 비장 초음파',
 '술 후 국소 초음파',
 '비장 종괴 초음파',
 'C spine MRI',
 'CT/MRI',
 'Brain MR+CT',
 '술부 피부 초음파',
 '직장 재주술후',
 '흉부 및 전지 방사선',
 '목뒤 초음파',
 '비장, 림프절 초음파',
 '후지 및 척추 방사선',
 '방광초음파',
 '임신확인 초음파',
 '전지 방사선 추가 촬영',
 '두개 방사선',
 '흉추 방사선',
 '술부 방사선',
 '위-십이지장 초음파',
 '우안 주변 초음파',
 '복부 및 피부 국소 초음파',
 '식도조영검사',
 '신우 천자',
 '종괴 초음파',
 'MR Findnings',
 '흉벽 초음파',
 '흉강 초음파',
 'Brain MRI',
 '비뇨-생식기계 초음파',
 '방광~요도 초음파',
 '간담도/ 소화기 초음파',
 '전,후지, 두부 방사선',
 '인후두부 방사선',
 '생식기 초음파',
 '담낭과 십이지장 초음파',
 '우측 침샘제거 술부 초음파',
 '회장 초음파',
 '서혜부와 음낭초음파',
 '비뇨기 초음파',
 '인후부두 방사선',
 '안와주변 초음파',
 '척추 MRI',
 '흉복부 전지 방사선',
 '담낭 초음파',
 '목종괴 초음파',
 '직장초음파',
 '술후 전지 방사선',
 '우측 경부 방사선',
 'OS안구 초음파',
 '비뇨기, 고환 초음파',
 'MR, CT',
 '목부위 무마취 CT',
 '전지 bone CT',
 '비뇨기, CVC초음파',
 '항문낭 초음파',
 '복부 및 골반 방사선',
 '좌측 SUB 술부 초음파',
 '림프절초음파',
 '배뇨 후 비뇨기 초음파',
 '태아 상태 초음파',
 '흉부 흡호기 방사선',
 '흉수 125 cc 천자후 평가',
 '우측 전지 방사선',
 '전지방사선-술후평가',
 '우측 경부 초음파',
 '흉수 20cc 천자',
 '유선부위 종괴 초음파',
 'cardiac CT',
 '유선 주변 초음파',
 '담낭, 췌장, 항문낭 초음파',
 '비뇨기, 비장 초음파',
 '담도. 췌장 초음파',
 '서헤부 초음파',
 '골반방사선',
 '간이 생식기계 초음파',
 '위 분문부 초음파',
 '둔부 초음파',
 '간. 담도 초음파',
 '비뇨기계 초음파',
 '비뇨기, 부신 초음파',
 '귀 종괴 초음파',
 '임신 확인 초음파',
 '교상부위 초음파',
 '천문 초음파',
 '후지 술부 초음파',
 '방광 주변 초음파',
 '하악아래 방사선',
 '비뇨 생식기 초음파',
 '흉부 및 두개부 방사선',
 '피하초음파',
 '우측 흉수 500ml 천자',
 '좌측 주관절 술후 방사선',
 '복부 방사선',
 '골격계방사선',
 '소화기, 술부 초음파',
 '심장/흉강 초음파',
 '두부초음파',
 '복부 확인 초음파',
 '국초 초음파',
 '하악밑 주변 초음파',
 '비뇨기 단일 초음파',
 '간, 비장 초음파',
 '방광과 복수 천자',
 '**위 가스 제거 후 초음파',
 'MR 1차',
 '위 , 십이지장 초음파',
 '위내시경후 소화기 초음파',
 'CT and MR',
 '심낭수 30ml 천자',
 '복부천자',
 '훙부 방사선',
 '흉수 300cc천자후 평가',
 '결장 초음파',
 '안구 초음파',
 '국소초음파',
 '흉부 . 전지 방사선',
 'Carpal/Phalanges 방사선',
 '좌측 후지 발목관절 방사선',
 '우측 후지 술후 방사선',
 '좌측 전지 술후 방사선',
 'FNA 후 국소 초음파',
 '흉요추 방사선',
 '흉수 250cc 천자후 방사선',
 'achilles tendon초음파',
 '혈전확인 간이 초음파',
 '혈관 초음파',
 '간담도 초음파',
 '탈장부위 초음파',
 '임신 초음파',
 '인후부방사선',
 '두개골 아래 초음파',
 '복부 간이 초음파',
 '방광방사선',
 '서혜부 및 생식기 초음파',
 '등쪽 종괴 초음파',
 '흉수 40cc천자후 평가',
 '임신초음파',
 '담낭-담도 초음파',
 '간담췌, 위장관 초음파',
 '간담췌, 방광 초음파',
 '* 흉수 천자',
 '상복부 국소 초음파',
 '간이초음파',
 '우안 안구 초음파',
 '흉.복수 천자',
 '좌측 흉부 피하 초음파',
 '흉ㅂ고부 방사선',
 '좌측 FHNO 술후 방사선',
 '간담도, 부신 초음파',
 '간담도, 술부 초음파',
 '전지 붕대후 방사선',
 '좌측 두경부 초음파',
 '흉부. 복부. 골반골 방사선',
 '간담도 방광 초음파',
 '담낭, 부신 초음파',
 '소화기/간담도 초음파',
 '심장 초음파',
 '골반 방사선',
 '간과 신장 초음파',
 '간담도 췌장 소화기 초음파',
 '견관절 방사선',
 '간, 소화기, 췌장 초음파',
 '직장 술부 초음파',
 '좌측 서혜부 초음파',
 '술부, 췌장 초음파',
 '림프절 재평가 초음파',
 '응급 흉부 초음파',
 '좌측 견관절 방사선',
 '간, 비장, 소화기 초음파',
 '간,비장 결절',
 '두경부 초음파',
 'CT 최종',
 '생식기계',
 '피부결절 초음파',
 '췌장과 부신 초음파',
 'MRI & CT',
 '복수 확인초음파',
 '후지 발목관절 방사선',
 '낭종 천자',
 '피부 초음파',
 '간실질과 간문맥 초음파',
 '흉수 160, 복수 190ml 천자',
 '전지 술후 방사선',
 '담낭. 생식기 초음파',
 '무마취 CT',
 '간, 부신 초음파',
 '담낭, 소화기 초음파',
 'MR, CT 1차',
 '배꼽 허니아 초음파',
 '간 및 비장 초음파',
 '전지 주관절',
 '흉수 천자-약 60ml 천자',
 '흉수천자',
 '복 초음파',
 '요로계 초음파',
 '담낭천자',
 '간 및 방광 초음파',
 '소화기, 생식기계 초음파',
 'MR/CT 최종',
 '전신 방사선',
 '혈전 관련 초음파',
 '지역병원 방사선 판독',
 '소화기 국소 초음파',
 'MR/CT',
 '위, 폐 초음파',
 '간이 혈관 초음파',
 '혈종',
 '흉수 방사선',
 '경부 방사선ㄱ',
 '상완골 방사선',
 '흉수 200ml 천자',
 'MR',
 '술 후 꼬리 방사선',
 '태아 확인 방사선',
 '술전 흉부 방사선',
 '위와 생식기 초음파',
 'FIP',
 '림프절 초음파',
 '대동맥 분지부 초음파',
 '미추 방사선',
 '흉요추-꼬리 방사선',
 '대동맥 초음파',
 '좌측 귀초음파',
 '소화기, 부신, 방광 초음파',
 '아킬레스건 초음파',
 '소화기 방사선',
 '응급흉부 초음파',
 '응급 초음파',
 '소화기 초음피',
 '비뇨생식기계 초음파',
 '담낭 및 방광 초음파',
 '골격방사선',
 '비장.비뇨기 초음파',
 '비뇨기, 항문 주위 국소초음파',
 '후지 척추 방사선',
 '전지 방사선 평가',
 '소화기, 간담도, 비장 초음파',
 'Skull CT',
 '경부 방사선',
 '직장 초음파',
 '우신초음파',
 '옆구리 종괴 초음파',
 '비뇨기, 종괴 초음파',
 '하악 아래 초음파',
 '서혜부 초음파',
 '연부조직 초음파',
 '좌측 전지 발가락 방사선',
 '간담췌초음파',
 '방광 및 좌측 부신 초음파',
 '담낭과 방광 초음파',
 '비뇨기, 생식기계 국소 초음파',
 '간. 담도 췌장 초음파',
 '흉복부, 두부 방사선',
 '경부초음파',
 '안과 초음파',
 '혈액성 흉수 18cc 천자',
 '췌장. 담도 초음파',
 '척추. 지골 방사선',
 '경부 방사선 재촬영',
 '술부, 방광 초음파',
 '간문 초음파',
 '흉부. 복부. 두개골 방사선',
 '좌측 폐 후엽 폐초음파',
 '방사선 재촬영',
 '회음부 초음파',
 '복부 초음퍼',
 '소장 술부 초음파',
 '복부 피하 초음파',
 '비장, 부신, 후복강 초음파',
 '* 요천자',
 '림프절, 담낭 초음파',
 '두부/척추 방사선',
 '흉요추방사선',
 '흉복수초음파',
 '우측 술부 초음파',
 'MR 1차 판독',
 '두부방사선',
 '안구초음파',
 '소화기, 림프절 초음파',
 '요추골 방사선',
 '우측 후지 지골 방사선',
 '기흉천자',
 'CT&MRI',
 '흉수 125cc 천자후 방사선',
 '간담도계, 췌장 국소 초음파',
 '두경부 방사선',
 '신장 초음파',
 '서혜부초음파',
 '두부 초음파',
 'Brain MR',
 '간주변 초음파',
 '술후 2주',
 'C and TL spine MRI',
 '간 담도, 소화기 초음파',
 '우측 겨드랑이 초음파',
 'MR Findigns',
 '우측 이도 술부 초음파',
 '복부 술후 방사선',
 'brain MRI',
 '술 후 초음파',
 '두개골 방사선',
 '간이 심장/폐 초음파',
 '부분 초음파',
 '간담췌, 소화기 초음파',
 '간, 간 혈관 초음파',
 '간,담도,췌장 초음파',
 '간비장 초음파',
 '흉수 150cc 천자후 평가',
 '좌측 후복강 초음파',
 '가스부위 초음파',
 '요도 방사선',
 '복수 확인 초음파',
 '흉수 130ml, 복수 140ml 천자',
 '위 초음파',
 '복부 초음파',
 '술후 방사선',
 'TL spine MRI',
 '후지 방사선',
 '흉복부. 척추 방사선',
 '목부위 초음파',
 '* 심낭수 천자',
 '간초음파',
 '담낭 및 비뇨기 초음파',
 '술부 피하 초음파',
 '비장, 비뇨기 초음파',
 '흉수 80ml 천자',
 '두개부 및 경추 방사선',
 '위, 담낭 초음파',
 '척추 방사선',
 '피부 술부 초음파',
 '비장, 부신 초음파',
 '하악 침샘 초음파',
 '골격계 방사선',
 '두개골방사선',
 '우안 초음파',
 '십이지장과 주변 초음파',
 '전지방사선',
 '두부 방사선',
 '피부 국소 초음파',
 '신탄진 방사선',
 '낭종 초음파',
 'MR. CT',
 '간, 담도계, 비뇨기 초음파',
 '간 및 비뇨기 초음파',
 '피하 국소 초음파',
 '전지 발목 관절 방사선',
 '머리 방사선',
 '고환초음파',
 'CT and MRI',
 '8월 13일 초음파',
 '유선 초음파',
 '배쪽 배꼽 허니아 초음파',
 '비뇨기계 국소 초음파',
 '방광과 전립샘 초음파',
 '전후지 발가락뼈 방사선',
 '- 흉수, 복수 300ml 천자',
 '고환집 초음파',
 'Bone CT 1차',
 '담낭, 비뇨기초음파',
 '우측 안구 초음파',
 '림프절과 회장 초음파',
 '태아 초음파',
 '좌측 주관절 방사선',
 '항문초음파',
 '전후방사선',
 '췌장초음파',
 '흉부 바사선',
 '소화기와 비뇨기 초음파',
 '비뇨기와 피하 종괴 초음파',
 'bone CT 3D',
 '* 흉수 130ml 천자',
 '간담췌 국소 초음파',
 '간. 소화기 초음파',
 '태아초음파',
 '하악 림프절 초음파',
 '간담도, 장간막 초음파',
 '흉복부 및 척추 방사선',
 '* 복수 천자',
 '두개부 방사선',
 '부갑상선 초음파',
 '우측 전 방사선',
 '간이 심장&폐 초음파',
 '신장, 췌장 초음파',
 '우측 후지 발가락 방사선',
 '담도계 초음파',
 '공장 병변 초음파',
 '좌측 후지 술후 방사선',
 '소화기+담낭 초음파',
 '흉부. 두개 방사선',
 '생식기계 초음파',
 '흉수 320 ml, 복수 270 ml 천자',
 '방광 요도 초음파',
 '두개골 술후 방사선',
 '골반골절 술후 방사선',
 '소화기, 비뇨기 초음파',
 '비뇨기+소화기 초음파',
 '복부 및 두개 방사선',
 '소장 초음파',
 '위장관계 초음파',
 '흉복수 천자',
 '복벽 초음파',
 '흉수 100ml, 복수 500ml 천자',
 '술부, 위장관 초음파',
 '부신초음파',
 '담낭절제술 후 초음파',
 '흉수 350cc 천자',
 '흉수 7cc 천자후 평가',
 '척추방사선',
 '간이 안구초음파',
 '소화기, 방광 초음파',
 '두개골. 경추 방사선',
 '복부 초음퍄',
 '둔부 방사선',
 'L spine MRI',
 '좌측 폐초음파',
 'MRi',
 '좌측 안구 초음파',
 '항문낭초음파',
 '장절개술 후 국소 초음파',
 '비뇨기, 복수 확인 초음파',
 '슬개골 방사선',
 '전립샘 초음파',
 '좌측 전지 방사선',
 '비장 초음파',
 '경추 MRI',
 '간담도, 방광 초음파',
 '간담췌 초음파',
 'PSS 술 후 초음파',
 '안와 주변 초음파',
 '담도 췌장 초음파',
 '미추방사선',
 '요추 척추 방사선',
 '비장초음파',
 '요관 결석위치 확인 초음파',
 '전지 방사선 : NRF',
 '간담도 복부 초음파',
 '경부, 소화기 초음파',
 '술 부 초음파',
 '60ml 천자',
 '복수 천자',
 '술부초음파',
 '*환자 호흡 불량으로 천자',
 '간문맥 혈관 초음파',
 '안구 후방 초음파',
 '흉수 약 160ml 천자',
 '응급 심장 초음파',
 '폐초음파',
 '우신 초음파',
 '하악림프절 초음파',
 '120ml 흉수천자',
 '간. 담도. 췌장 초음파',
 '침샘부위 초음파',
 '중성화 수술 술부 초음파',
 '복수 천자- 460ml',
 '좌측 전지 종괴 초음파',
 '우측 MPL술부 초음파',
 '흉요추 척추 방사선',
 '위장관 간이 초음파',
 '흉수 천자후 재평가',
 '방광 및 요도 초음파',
 '흉복부. 두개 방사선',
 '우안 안구적출부위 초음파',
 '담낭.방광 초음파',
 '심장 국소 초음파',
 '술부 초음파',
 '복수 초음파',
 '흉부 VD',
 '술부, 소화기 초음파',
 '국소 피하 초음파',
 '간담도, 비장 초음파',
 'MRI',
 '방광 단일 초음파',
 '턱밑 초음파',
 '비뇨기, 췌장 국소 초음파',
 '흉복부. 후지 및 꼬리 방사선',
 '술 후 방사선',
 '방광, 담낭 초음파',
 '고정기구 제거후 방사선',
 '혈관초음파',
 '요도 주변 초음파',
 '요천추 방사선',
 '복수 양 확인초음파',
 '대동백 분지부와 심장 응급 간이 초음파',
 '하악아래 림프절 초음파',
 '하부 비뇨기 초음파',
 '술부 주변 초음파',
 '복수 천자- 500ml',
 'PSS 여부 초음파',
 '비장, 소화기 초음파',
 '두개부 초음파',
 '흉복부. 두개방사선',
 '비장, 신장 초음파',
 '담낭 초으파',
 '간담도/소화기 초음파',
 '간이 Echo, 폐 초음파',
 '흉부 초음파',
 '췌장 초음파',
 '소화기계 국소 초음파',
 '우측 전지방사선',
 '흉부-인후두부 방사선',
 '방사선평가',
 '신장초음파',
 '요도 초음파',
 '간, 소화기 초음파',
 '경추 방사선',
 'SUB 술 후 비뇨기 초음파',
 '술 후 전지 방사선',
 '두개골 척추 방사선',
 '흉수 50 ml 천자',
 '안초음파',
 '전립선 초음파',
 '위장관 초음파',
 '간담도, 소화기, 방광 초음파',
 '비뇨기, 소화기 초음파',
 '항문 우측 엉덩이쪽 초음파',
 '비뇨기 간이 초음파',
 'MR and CT',
 '서혜부 및 요도 초음파',
 '상복부와 흉부초음파',
 '피하 초음파',
 '비뇨기간이초음파',
 '전립샘. 방광 초음파',
 '골반골 bone CT',
 '방광 천자',
 '혈전 초음파',
 '간. 혈관 초음파',
 '뇌실초음파',
 '후지 발가락 방사선',
 '복수 확인과 항문낭 초음파',
 '흉부 방사선',
 '좌안 초음파',
 '항문아래 연부조직 초음파',
 'abdomen CT',
 '흉강 CT',
 '방광 국소 초음파',
 '신우 조영 후 배출되지 않아 천자',
 '경부 초음파',
 '비뇨기 및 소화기 초음파',
 '흉부 술전 방사선',
 'CT 1차 간이',
 '방광 생식기 초음파',
 '간담도, 소화기 초음파',
 '흉복수천자',
 '소화기 초음파',
 '우측 주관절 술후 방사선',
 '꼬리방사선',
 '비뇨 생식기계 초음파',
 '소화기초음파',
 '소화기, 림프절 국소 초음파',
 '하악방사선',
 '경추 초음파',
 '담도 초음파',
 '*흉복수 비교용 초음파',
 '비뇨기 국소 초음파',
 '두개골. 척추 방사선',
 '복수 확인차 초음파',
 'Pelvic CT',
 '십이지장 초음파',
 '유선 종괴 초음파',
 '하악밑 림프절 초음파',
 '흉수 120cc천자후 평가',
 '흅부 방사선',
 '좌측 전지방사선',
 '음경과 요도 주변 술부와 비장 초음파',
 '경추방사선',
 '담, 췌, 비장 초음파',
 'CT 간이',
 '흉복부. 골격계 방사선',
 '종괴초음파',
 '소화기계 초음파',
 '하악밑 초음파',
 '소화기 및 술부 초음파',
 '담낭과 소장 초음파',
 '결장초음파',
 '흉수 42ml 천자',
 '복수 270ml 정도 천자',
 '요추하 림프절 초음파',
 '소화기/비뇨기 국소 초음파',
 '400ml 천자',
 '비뇨생식기 초음파',
 '췌장. 비뇨기 초음파',
 'R-PDA ductus초음파',
 '술후 초음파',
 '림프절, 피하종괴 초음파',
 '좌측 상복부',
 '림프절 f/u 초음파',
 '담낭, 비뇨기 초음파',
 '흉부. 골격계 방사선',
 '생식기 방광 초음파',
 '항문주변 초음파',
 'OS 안구 초음파',
 '흉부 및 전지방사선',
 '상복부 부분 초음파',
 '간, 복수 확인 초음파',
 '후지 초음파',
 '두개골 초음파',
 '고정기구 일부 제거후 방사선',
 '심낭수 확인 초음파',
 '소화기, 생식기 초음파',
 '좌안 안구 초음파',
 '국소 술부 초음파',
 '담도계, 췌장 초음파',
 '흉부 주변 초음파',
 '간담도, 신장 초음파',
 '경부 주변 방사선',
 '휴부 방사선',
 '비뇨. 소화기 초음파',
 '심장, 흉부 초음파',
 '소화기 단일 초음파',
 'TL spine MR',
 '인후두부 초음파',
 '골반골 방사선',
 '간담도 및 소화기 초음파',
 '복부 초엄파',
 '자궁 초음파',
 '전자 방사선',
 '보굽 방사선',
 '흉,복수 천자',
 '비강주변 초음파',
 '간.비장 초음파',
 '우측 전지 술후 방사선',
 '후지 술 후 방사선',
 'CT',
 '간 초음파',
 '태아 확인 초음파',
 '척추/고관절 방사선',
 '뇌실 초음파',
 '흉부. 서혜부 방사선',
 '간담도초음파',
 '흉골 방사선',
 '간과 담도계 초음파',
 '흉부초음파',
 '비장 제거 술부 초음파',
 '간. 비장 초음파',
 '간이 초음파',
 '비뇨기초음파',
 'Brain and TL MRI',
 '단일 방광 초음파',
 '관절 초음파',
 '경부방사선',
 '간담낭 초음파',
 '꼬리뼈 방사선',
 '술부주위 초음파',
 '흉수 확인 및 천자',
 '우측 서혜부 초음파',
 '간/혈관 초음파',
 '간부신 초음파',
 '흉수 천자후 평가',
 '두개골. 후지. 전지 방사선',
 '비장과 신장 초음파',
 '피부초음파',
 '복수천자',
 '항문 초음파',
 '술후 국소 초음파',
 '흉수 200cc 천자',
 '흉수 천자후 방사선',
 '흉수 천자',
 '담도초음파',
 '구토 처치후 초음파',
 '28173 쿠키 방사선',
 '턱아래 초음파',
 '심낭수 초음파',
 '두개골 CT',
 '비뇨기. 비장 초음파',
 '우측 지골 골절 술후 방사선',
 '소화기+간담낭 초음파',
 '우측 신장 초음파',
 '흉부. 전지 방사선',
 '방광, 술부 국소 초음파',
 '사지 방사선',
 '외이도 부분 초음파',
 '췌장 담도 초음파',
 '흉수- 80ml 천자',
 '간담도, 위 초음파',
 '대동맥 분치부 초음파',
 'CT, MR',
 '외부 CT 판독',
 '요천추방사선',
 '주관절 방사선',
 '간담도계 초음파',
 'CT, MRI 최종',
 '전지. 경추 방사선',
 '심낭수 천자',
 '소화기 및 연부조직 초음파',
 '안면초음파',
 '우측 전지 초음파',
 '소화기/췌장 초음파',
 '비뇨기 방사선',
 '- 흉수천자',
 '항문낭 술부 초음파',
 '간이 echo / 폐 초음파',
 '단일 초음파',
 '하악아래 초음파',
 '간혈관 초음파',
 '요추방사선',
 '종괴 국소 초음파',
 '경부 국소초음파',
 '번지 방사선',
 '근육간이초음파',
 '전지 초음파',
 '간담도/췌장 초음파',
 '식도 이물 처치 후 방사선',
 '방광. 회음부 초음파',
 '흉복부 골격계 방사선',
 '좌안 적출부위 초음파',
 '우측 견관절',
 '내시경 후 방사선',
 '폐 초음파',
 '유선종괴 초음파',
 '후지 처치후 방사선',
 '흉부 및 두개 방사선',
 '술 부 국소 초음파',
 '간담췌, 종괴 초음파',
 '응급초음파',
 '비뇨기, 장 초음파',
 '생식기초음파',
 '방광 초음파',
 '좌신 초음파',
 '병변부 초음파',
 '항문 주변 초음파',
 'TL MRI',
 'MRI and CT',
 '위초음파',
 '하악주변 초음파',
 '후지종괴 초음파',
 '부신 초음파',
 '고관절 방사선',
 '후복강 초음파',
 '국소 초음파',
 '비뇨기계, 부신 초음파',
 '비장제거 술부 초음파',
 '전지 방사선',
 '좌측 후지 발목관절',
 '간담도 췌장 초음파']

['술부, 비뇨기 초음파',
 '후지 술후 방사선',
 '간 담도 초음파',
 '비장 절제 부위 초음파',
 '소화기+대동맥 초음파',
 '비뇨기. 회음부 초음파',
 '간담도, 비뇨기 초음파',
 '방광천자',
 '간이 흉강 초음파',
 '방광결석 제거술후 방사선',
 '꼬리 방사선',
 '우심 초음파',
 '소화기, 췌장 초음파',
 '인후두 방사선',
 '복부간이초음파',
 '술후 간담도 초음파',
 '소화기, 신장 초음파',
 '피하 병변 초음파',
 '항문 주위 초음파',
 '결절 초음파',
 '간, 위 초음파',
 'SUB 제거 후 비뇨기 초음파',
 '주폐동맥 초음파',
 '두개골 인후두 방사선',
 '술부 국소 초음파',
 '단일 장기 초음파',
 '침샘 초음파',
 '지역병원 방사선 평가',
 '요도방사선',
 '술후 소화기 초음파',
 '간이 소화기 초음파',
 '좌측 견관절 술후 방사선',
 '간담도',
 '흉부, 인후두부 방사선',
 '간, 복수 초음파',
 'MR 최종',
 '비뇨기 생식기 초음파',
 '전지 발가락 방사선',
 '방광 술후 초음파',
 '우측 후지 초음파',
 '술부 및 소화기 초음파',
 '횡격막 파열 술 후 방사선',
 '우측 후지 발가락 술후 방사선',
 '좌측 전지 발가락 골절',
 '하악아래 림프절 주변 초음파',
 '뇨관결석 초음파',
 '비뇨 생식기, 부신 초음파',
 'TL MR',
 '좌측 요관 결석',
 '잠복고환 초음파',
 '전지 관절 초음파',
 '요추 방사선',
 '흉수 약 110ml 천자',
 '자궁초음파',
 'MRI 차',
 '좌측 후지 초음파',
 '요척골 골절 술후 방사선',
 'Abdomen CT',
 '간이 폐초음파',
 '장초음파',
 '흉골과 좌측 상완골 주변 초음파',
 '복수 확인, 항문낭 초음파',
 '견갑 방사선',
 '담낭초음파',
 '간, 비장과 비뇨기 초음파',
 '흉수 약 35cc천자',
 '생식기 단일 초음파',
 '담도.췌장 초음파',
 '담낭, 비장 초음파',

In [166]:
import os

file_name = file_path + "ah2/testlist.txt"
f = open(file_name, 'w')

test = []

while True:
    test_name = f.readline()
    if not test_name:
        break
    test.append(test_name)

f.close()

In [157]:
for i in range(len(ah2_test)):
    ah2_test['TestName'][i] = ""
    testlist = []
    for j in range(len(test)):
        if test[j] in ah2_test['study_interpretation'][i]:
            testlist.append(test[j])
    testlist = list(set(testlist))
    testlist = ", ".join(testlist)
    ah2_test['TestName'][i] = testlist

C:\Users\USER\AppData\Local\Temp\ipykernel_23232\1484327388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ah2_test['TestName'][i] = ""
C:\Users\USER\AppData\Local\Temp\ipykernel_23232\1484327388.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ah2_test['TestName'][i] = testlist
C:\Users\USER\AppData\Local\Temp\ipykernel_23232\1484327388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ah2_test['TestName'][i] = ""
C:\Users\U

In [161]:
ah2_test = ah2_test.drop(['study_desc', 'study_interpretation'], axis=1)

ah2_test

,pet_serial,pet_name,study_date,TestName
0,194,사랑,2016-09-10 0:00,후지 방사선
1,361,모리,2016-09-11 0:00,"복부 초음파, 흉부 방사선"
2,466,가지,2016-09-16 0:00,후지 방사선
3,544,꾹이,2016-09-23 0:00,흉부 방사선
4,42,심슨,2016-09-19 0:00,흉부 방사선
...,...,...,...,...
72950,16657,루비,2024-07-31 21:18,
72951,16657,루비,2024-07-31 21:12,
72952,32822,잼,2024-07-31 22:21,"복부 방사선, 흉부 방사선"
72953,24202,앵두,2024-07-31 23:22,복부 방사선


In [151]:
ah2_test['TestName'].value_counts()

TestName
흉부 방사선                             15798
                                   10154
복부 초음파                              7992
복부 방사선                              7114
후지 방사선                              5430
                                   ...  
CT, MR, MRI  and CT , MRI              1
MR, 흉부 방사선, 경추 방사선, MRI, 요추 방사선        1
복부 방사선, 흉부 방사선, 심낭수 천자                 1
흉수 천자, 흉부 방사선, 심낭수 천자                  1
사지 방사선 , 사지 방사선                        1
Name: count, Length: 2487, dtype: int64

In [170]:
ah2_test = ah2_test.rename(columns={'study_date':'chart_date'})
for i in range(len(ah2_test)):
    ah2_test['chart_date'][i] = ah2_test['chart_date'][i].split()[0]
    
ah2_test_con = pd.concat([ah2_blood, ah2_test])

ah2_test_con.rename(columns={'pet_serial':'Pet.id'}, inplace=True)
ah2_test_con.rename(columns={'pet_name':'Pet.petName'}, inplace=True)
ah2_test_con.rename(columns={'chart_date':'diagnosisDateTime'}, inplace=True)

In [ ]:
ah2_df = pd.merge(ah2_df, ah2_test_con, on=['Pet.id', 'diagnosisDateTime'], how='outer')

In [ ]:
embedding_df = pd.concat([ah1_df, ah2_df])

In [ ]:
embedding_df = embedding_df.reset_index(drop=True)

embedding_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label,chartdesc_title
0,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.910,<p>S</p>\n<p>건강했음.</p>\n<p>어제 침을 너무 많이 흘린 것 이외...,0,NaN
1,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.600,<p>S</p>\n<p>- 특이사항 없었음</p>\n<p>O</p>\n<p>- 오른...,0,NaN
2,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.380,<p>S</p>\n<p>더 나아지지는 않는 것 같다고 말씀</p>\n<p>활력은 양...,0,NaN
3,,2022-02-07T08:12:32.807Z,[],[],[],[],1347,아롱이,SF,3.380,<p></p>,0,NaN
4,심장사상충,2022-02-07T07:48:12.477Z,[],[],[],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",1907,하루,SF,4.595,<p>S</p>\n<p>심장사상충 예방하기 위해 내원</p>\n<p>최근에 사료토<...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196672,"백신부작용 후 장염, 췌장염가능성",2024-07-31,NaN,NaN,NaN,NaN,16657,루비,NaN,NaN,S)\r\n- 2-3일 멎었다가 다시 시작\r\n- 그제 설사 1-2번\r\n- 어...,0,
196673,간식 먹고 켁켁거림,2024-07-31,NaN,NaN,NaN,NaN,32822,잼,NaN,NaN,"S)\r\n30분 전에 커다란 간식 하나 통쨰로 삼킴\r\n구토 1회 물 토, 차에...",0,
196674,초콜릿 섭취(1시간 반전)//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,24202,앵두,NaN,NaN,S)\r\n잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원\r\n?\r\n관리...,0,
196675,호흡 가쁨//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,30604,베니,NaN,NaN,S)\r\n평소에 간헐적인 기침 거위소리는 있었는데\r\n조금전 산책하고 들어와서 ...,0,


# Data load

In [57]:
import re

def clean(text:str):
    not_use = re.compile('[ㄱ-ㅎㅏ-ㅣ]+')
    dup_space = re.compile('[ \t]+')
    dup_stop = re.compile('[\.]+')

    cleaned = not_use.sub('', text.strip())
    cleaned = dup_space.sub(' ', cleaned)
    cleaned = dup_stop.sub('.', cleaned)

    return cleaned

embedding_df['Subjective.visibleText'] = embedding_df['Subjective.visibleText'].apply(lambda text: text.replace('<p>', ' ').replace('</p>', ' ').replace('<br>', ' ').replace('\n', ' ').replace('\r', ' ').replace('<p style="color: rgb(255, 0, 0);">', ' '))
embedding_df['Subjective.visibleText'] = embedding_df['Subjective.visibleText'].apply(clean)

embedding_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label,chartdesc_title
0,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.910,S 건강했음. 어제 침을 너무 많이 흘린 것 이외 특이사항 없었음. O 유치 빠지는...,0,NaN
1,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.600,S - 특이사항 없었음 O - 오른쪽 슬개골탈구 g2 A - HG + FL,0,NaN
2,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.380,S 더 나아지지는 않는 것 같다고 말씀 활력은 양호함. 걸으려고 하면 다리가 벌어지...,0,NaN
3,,2022-02-07T08:12:32.807Z,[],[],[],[],1347,아롱이,SF,3.380,,0,NaN
4,심장사상충,2022-02-07T07:48:12.477Z,[],[],[],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",1907,하루,SF,4.595,S 심장사상충 예방하기 위해 내원 최근에 사료토 O 우측 항문낭만 차있었음. 이빨 ...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196672,"백신부작용 후 장염, 췌장염가능성",2024-07-31,NaN,NaN,NaN,NaN,16657,루비,NaN,NaN,S) - 2-3일 멎었다가 다시 시작 - 그제 설사 1-2번 - 어제 설사 다량 3...,0,
196673,간식 먹고 켁켁거림,2024-07-31,NaN,NaN,NaN,NaN,32822,잼,NaN,NaN,"S) 30분 전에 커다란 간식 하나 통쨰로 삼킴 구토 1회 물 토, 차에서 구역질은...",0,
196674,초콜릿 섭취(1시간 반전)//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,24202,앵두,NaN,NaN,S) 잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원 ? 관리받는 질환 없음 ...,0,
196675,호흡 가쁨//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,30604,베니,NaN,NaN,S) 평소에 간헐적인 기침 거위소리는 있었는데 조금전 산책하고 들어와서 집에서 갑자...,0,


In [58]:
short_index = embedding_df[(embedding_df['Subjective.visibleText'].map(len) < 30)].index

In [59]:
len(short_index)

16826

In [60]:
embedding_df = embedding_df.drop(short_index)

embedding_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label,chartdesc_title
0,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.910,S 건강했음. 어제 침을 너무 많이 흘린 것 이외 특이사항 없었음. O 유치 빠지는...,0,NaN
1,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.600,S - 특이사항 없었음 O - 오른쪽 슬개골탈구 g2 A - HG + FL,0,NaN
2,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.380,S 더 나아지지는 않는 것 같다고 말씀 활력은 양호함. 걸으려고 하면 다리가 벌어지...,0,NaN
4,심장사상충,2022-02-07T07:48:12.477Z,[],[],[],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",1907,하루,SF,4.595,S 심장사상충 예방하기 위해 내원 최근에 사료토 O 우측 항문낭만 차있었음. 이빨 ...,0,NaN
5,접종,2022-02-07T07:41:31.560Z,['강아지 - 심장사상충 키트 검사 (HW Ag kit)'],[],[],"['AA1.DHPP', 'AB2.켄넬코프', 'AD1.광견병(수입)']",1906,배추,CM,4.350,S - 동생 분 강아지인데 추가접종 때문에 내원하심 - 심장사상충 히스토리 불분명 ...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196672,"백신부작용 후 장염, 췌장염가능성",2024-07-31,NaN,NaN,NaN,NaN,16657,루비,NaN,NaN,S) - 2-3일 멎었다가 다시 시작 - 그제 설사 1-2번 - 어제 설사 다량 3...,0,
196673,간식 먹고 켁켁거림,2024-07-31,NaN,NaN,NaN,NaN,32822,잼,NaN,NaN,"S) 30분 전에 커다란 간식 하나 통쨰로 삼킴 구토 1회 물 토, 차에서 구역질은...",0,
196674,초콜릿 섭취(1시간 반전)//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,24202,앵두,NaN,NaN,S) 잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원 ? 관리받는 질환 없음 ...,0,
196675,호흡 가쁨//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,30604,베니,NaN,NaN,S) 평소에 간헐적인 기침 거위소리는 있었는데 조금전 산책하고 들어와서 집에서 갑자...,0,


In [61]:
embedding_df = embedding_df.reset_index(drop=True) 

embedding_df

,chiefComplaint,diagnosisDateTime,PlanRxCode,PlanSuppliesCode,PlanTxCode,PetVaccineOnSameDate,Pet.id,Pet.petName,Pet.petGender,Pet.recentBodyWeightValue,Subjective.visibleText,Label,chartdesc_title
0,심장사상충,2022-02-07T09:14:23.727Z,[],[],[],"['BN1.넥스가드 스펙트라 2~3.5kg,ea']",1601,우유,F,1.910,S 건강했음. 어제 침을 너무 많이 흘린 것 이외 특이사항 없었음. O 유치 빠지는...,0,NaN
1,심장사상충,2022-02-07T09:03:25.600Z,[],[],[],"['BB1.하트가드 Blue', 'BC1.프론트라인 10kg 이하']",5,마리,F,8.600,S - 특이사항 없었음 O - 오른쪽 슬개골탈구 g2 A - HG + FL,0,NaN
2,재진,2022-02-07T08:31:09.877Z,['내복약 1일 2회(5kg 미만)'],"['관절보조제_PHYCOX', '시그니처바이_p/a_반려견_개당', '시그니처바이_...","['진료비_재진', '특수약 추가)gabapentin']",[],1347,아롱이,SF,3.380,S 더 나아지지는 않는 것 같다고 말씀 활력은 양호함. 걸으려고 하면 다리가 벌어지...,0,NaN
3,심장사상충,2022-02-07T07:48:12.477Z,[],[],[],"['BN3.넥스가드 스펙트라 3.5~7.5kg, ea']",1907,하루,SF,4.595,S 심장사상충 예방하기 위해 내원 최근에 사료토 O 우측 항문낭만 차있었음. 이빨 ...,0,NaN
4,접종,2022-02-07T07:41:31.560Z,['강아지 - 심장사상충 키트 검사 (HW Ag kit)'],[],[],"['AA1.DHPP', 'AB2.켄넬코프', 'AD1.광견병(수입)']",1906,배추,CM,4.350,S - 동생 분 강아지인데 추가접종 때문에 내원하심 - 심장사상충 히스토리 불분명 ...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179846,"백신부작용 후 장염, 췌장염가능성",2024-07-31,NaN,NaN,NaN,NaN,16657,루비,NaN,NaN,S) - 2-3일 멎었다가 다시 시작 - 그제 설사 1-2번 - 어제 설사 다량 3...,0,
179847,간식 먹고 켁켁거림,2024-07-31,NaN,NaN,NaN,NaN,32822,잼,NaN,NaN,"S) 30분 전에 커다란 간식 하나 통쨰로 삼킴 구토 1회 물 토, 차에서 구역질은...",0,
179848,초콜릿 섭취(1시간 반전)//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,24202,앵두,NaN,NaN,S) 잠깐 집 비운 사이 마일로를 하나 전부 섭취하여 내원 ? 관리받는 질환 없음 ...,0,
179849,호흡 가쁨//야고//3층대고,2024-07-31,NaN,NaN,NaN,NaN,30604,베니,NaN,NaN,S) 평소에 간헐적인 기침 거위소리는 있었는데 조금전 산책하고 들어와서 집에서 갑자...,0,


# Create Collection

In [2]:
DATASET = 'PetEMR'
MODEL = 'jhgan/ko-sroberta-multitask'
COLLECTION_NAME = 'PetEMR_db'
VECTOR_DIMENSION = 768
HOST = 'localhost'
PORT = '19530'
INDEX_TYPE = 'IVF_FLAT'

connections.connect(host=HOST, port=PORT)

EMBEDDING_FIELD_NAME = 'soap_embedding'

In [67]:
if utility.has_collection(COLLECTION_NAME):
    collection = Collection(name=COLLECTION_NAME)
    collection.drop()
    print(f'Delete collection: {COLLECTION_NAME}')

field2args = {
    'index': {'dtype': DataType.INT64, 'is_primary': True, 'auto_id': True},
    'complaint': {'dtype': DataType.VARCHAR, 'max_length': 5000},
    'datetime': {'dtype': DataType.VARCHAR, 'max_length': 30},
    'pet_id': {'dtype': DataType.INT64, 'max_length': 100},
    'pet_name': {'dtype': DataType.VARCHAR, 'max_length': 100},
    # 'pet_sex': {'dtype': DataType.VARCHAR, 'max_length': 30},
    # 'pet_weight': {'dtype': DataType.FLOAT, 'max_length': 50},
    'soap': {'dtype': DataType.VARCHAR, 'max_length': 50000},
    'test_name' : {'dtype': DataType.VARCHAR, 'max_length': 5000},
    'label': {'dtype': DataType.INT64, 'max_length': 1},
    EMBEDDING_FIELD_NAME: {'dtype': DataType.FLOAT_VECTOR, 'dim': VECTOR_DIMENSION}
}

In [68]:
for i in range(len(embedding_df)):
    if type(embedding_df['Pet.petName'][i]) == float:
        embedding_df['Pet.petName'][i] = str(embedding_df['Pet.petName'][i])

C:\Users\USER\AppData\Local\Temp\ipykernel_3664\3660593101.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  embedding_df['Pet.petName'][i] = str(embedding_df['Pet.petName'][i])
C:\Users\USER\AppData\Local\Temp\ipykernel_3664\3660593101.py

In [69]:
for i in range(len(embedding_df)):
    if type(embedding_df['chiefComplaint'][i]) == float:
        embedding_df['chiefComplaint'][i] = str(embedding_df['chiefComplaint'][i])

In [71]:
fields = [
    FieldSchema(name = field_name, **args)
    for field_name, args in field2args.items()
]

schema = CollectionSchema(fields=fields, description="Collection for PetEMR")
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [73]:
if collection.has_index():
    try:
        collection.drop_index()
    except:
        collection.realease()
        collection.drop_index()

In [12]:
index_params = {
    'metric_type': 'IP',
    'index_type': INDEX_TYPE,
    'params': {'nlist': 1}
}

In [75]:
collection.create_index(field_name=EMBEDDING_FIELD_NAME, index_params=index_params)
collection.load()
collection

<Collection>:
-------------
<name>: PetEMR_db
<description>: Collection for PetEMR
<schema>: {'auto_id': True, 'description': 'Collection for PetEMR', 'fields': [{'name': 'index', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'complaint', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'datetime', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 30}}, {'name': 'pet_id', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'pet_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'soap', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 50000}}, {'name': 'label', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'soap_embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': False}

# Use Embedding Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

C:\Users\USER\anaconda3\envs\sbert\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Tokenize sentences
def embedder(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])

    return sentence_embedding

In [79]:
sentences = embedding_df['Subjective.visibleText'].tolist()

In [80]:
sentences

['S 건강했음. 어제 침을 너무 많이 흘린 것 이외 특이사항 없었음. O 유치 빠지는 중 A/P 넥스가드',
 'S - 특이사항 없었음 O - 오른쪽 슬개골탈구 g2 A - HG + FL',
 'S 더 나아지지는 않는 것 같다고 말씀 활력은 양호함. 걸으려고 하면 다리가 벌어지는 양상 고개를 계속 왼쪽으로 갸우뚱 O Lt. head tilt Rt. unilateral UMN 왼쪽 뒷다리 medial buttress sign + 연발음 A/P Brain or neck 부위의 문제라고 설명 내복약 7일 : metho 22mg/kg, gaba 10mg/kg, famo 0.5mg/kg BID',
 'S 심장사상충 예방하기 위해 내원 최근에 사료토 O 우측 항문낭만 차있었음. 이빨 전반적으로 치석 A/P 넥스가드',
 'S - 동생 분 강아지인데 추가접종 때문에 내원하심 - 심장사상충 히스토리 불분명 O - 치석 매우 많음 - 슬개골탈구 소견 없음 - CHW Ag test: no antigen found A - 심장사상충 예방은 따로 하실 것',
 'S 그저께 밤에 새벽에 토를 시작 그 뒤부터는 먹는 것 다 구토. 집에서 키우는 식물 이파리가 구토에서 나왔음. 사료, 강아지 간식만 먹음. 말린 망고 먹었음. 이파리, 사료 외적으로 FB섭취 가능성은 낮음 O 복부 전체적으로 복압상승 severe한 치석 murmur G1-2 cPL < 10 chem, NRF A/P 항구토제 주사 SQ 내복약 3일 : metro 15mg/kg, famo 0.5mg/kg, meto 0.3mg/kg, 부스코판1T',
 'S - 잘 지냈는데 토요일날부터 많이 긁기 시작함 - 발을 핥고 귀를 간지러워함 O - 지간염 - 왼쪽 귓바퀴 부종 - 왼쪽 슬개골탈구 g1 A - 사이토포인트',
 'S - 미용을 2주전 다시 했는데 바닥에 항문을 끌고 다님 - 귀는 가끔씩 텀 O - 귀는 괜찮음 A - pds 1mg/kg SC - 추후에 미용 전에 알러지 약 처방 받으실 것',
 '오늘 처음으로 아침 약을

In [81]:
from torch.utils.data import DataLoader

# Create a DataLoader
batch_size = 256  # Reduce the batch size if memory issues persist
dataloader = DataLoader(sentences, batch_size=batch_size, shuffle=False)

all_embeddings = []

for batch in dataloader:
    encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    all_embeddings.append(sentence_embeddings)

# Concatenate all embeddings
all_embeddings = torch.cat(all_embeddings)

print("Sentence embeddings:")
print(all_embeddings)


Sentence embeddings:
tensor([[-0.1778, -0.2843,  0.1059,  ..., -0.1355,  0.1451, -0.2512],
        [-0.2948,  0.4724,  0.4211,  ..., -0.2196, -0.3139, -0.2023],
        [-0.3097,  0.0379, -0.2162,  ..., -0.0745,  0.2114,  0.1901],
        ...,
        [-0.4113, -0.2219, -0.1197,  ...,  0.1421,  0.3640, -0.1724],
        [-0.3211, -0.1756, -0.2531,  ..., -0.2533,  0.3344, -0.2190],
        [-0.3126, -0.4866, -0.2799,  ...,  0.1057,  0.1209, -0.1914]])


In [191]:
print("Sentence embeddings:")
print(all_embeddings)

Sentence embeddings:
tensor([[-0.1778, -0.2843,  0.1059,  ..., -0.1355,  0.1451, -0.2512],
        [-0.2948,  0.4724,  0.4211,  ..., -0.2196, -0.3139, -0.2023],
        [-0.3097,  0.0379, -0.2162,  ..., -0.0745,  0.2114,  0.1901],
        ...,
        [-0.4113, -0.2219, -0.1197,  ...,  0.1421,  0.3640, -0.1724],
        [-0.3211, -0.1756, -0.2531,  ..., -0.2533,  0.3344, -0.2190],
        [-0.3126, -0.4866, -0.2799,  ...,  0.1057,  0.1209, -0.1914]])


In [190]:
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

sentence_embeddings = embedder(sentences)

print("Sentence embeddings:")
print(sentence_embeddings)

KeyboardInterrupt: 

In [193]:
all_embeddings.shape

torch.Size([263007, 768])

# Insert

In [82]:
column2field = {
    'chiefComplaint': 'complaint',
    'diagnosisDateTime': 'datetime',
    'Pet.id': 'pet_id',
    'Pet.petName': 'pet_name',
    # 'Pet.petGender': 'pet_sex',
    # 'Pet.recentBodyWeightValue': 'pet_weight',
    'Subjective.visibleText': 'soap',
    'TestName': 'test_name',
    'Label': 'label'
}

field2column = {field: column for column, field in column2field.items()}

In [83]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe, embeddings, field2column, embedding_field):
        self.dataframe = dataframe
        self.embeddings = embeddings
        self.field2column = field2column
        self.embedding_field = embedding_field

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        items = {field: self.dataframe.iloc[idx][column] for field, column in self.field2column.items()}
        items[self.embedding_field] = self.embeddings[idx].tolist()
        
        return items

dataset = CustomDataset(embedding_df, all_embeddings, field2column, EMBEDDING_FIELD_NAME)

In [84]:
from tqdm.auto import tqdm

for data in tqdm(iter(dataset), total=len(dataset)):
    collection.insert([data])

collection.flush()
collection.num_entities

100%|█████████████████████████████████████████████████████████████████████████| 179851/179851 [10:32<00:00, 284.46it/s]


179851

# Upsert

In [ ]:
DATASET = 'PetEMR'
MODEL = 'jhgan/ko-sroberta-multitask'
COLLECTION_NAME = 'PetEMR_db'
VECTOR_DIMENSION = 768
HOST = 'localhost'
PORT = '19530'
INDEX_TYPE = 'IVF_FLAT'

connections.connect(host=HOST, port=PORT)

EMBEDDING_FIELD_NAME = 'soap_embedding'

In [ ]:
if utility.has_collection(COLLECTION_NAME):
    collection = Collection(name=COLLECTION_NAME)

In [ ]:
index_params = {
    'metric_type': 'IP',
    'index_type': INDEX_TYPE,
    'params': {'nlist': 1}
}

In [ ]:
collection.load()
collection

<Collection>:
-------------
<name>: PetEMR_db
<description>: Collection for PetEMR
<schema>: {'auto_id': True, 'description': 'Collection for PetEMR', 'fields': [{'name': 'index', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'complaint', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'datetime', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 30}}, {'name': 'pet_id', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'pet_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'soap', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 50000}}, {'name': 'label', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'soap_embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': False}

In [ ]:
column2field = {
    'chiefComplaint': 'complaint',
    'diagnosisDateTime': 'datetime',
    'Pet.id': 'pet_id',
    'Pet.petName': 'pet_name',
    # 'Pet.petGender': 'pet_sex',
    # 'Pet.recentBodyWeightValue': 'pet_weight',
    'Subjective.visibleText': 'soap',
    'TestName': 'test_name',
    'Label': 'label'
}

field2column = {field: column for column, field in column2field.items()}

In [ ]:
dataset = CustomDataset(embedding_df, all_embeddings, field2column, EMBEDDING_FIELD_NAME) 

In [ ]:
for data in tqdm(iter(dataset), total=len(dataset)):
    collection.insert([data])

collection.flush()
collection.num_entities

# Query

In [3]:
query = '기침이 너무 많이 나요. 밤에 더 심해요.'

query

'기침이 너무 많이 나요. 밤에 더 심해요.'

In [205]:
query = '설사가 심해요. 밥을 잘 못 먹어요.'

query

'설사가 심해요. 밥을 잘 못 먹어요.'

In [8]:
query_embedding = embedder(query)

#query_embedding = query_embedding.tolist()
query_embedding = query_embedding.squeeze(0)
print(query_embedding.shape)

query_embedding = query_embedding.tolist()

torch.Size([768])


In [15]:
query_embedding

[-0.2001143991947174,
 -0.22087574005126953,
 -0.44873252511024475,
 -0.16425052285194397,
 -0.01536857895553112,
 0.12426413595676422,
 -1.2782307863235474,
 0.725191593170166,
 0.10788028687238693,
 -0.35739263892173767,
 -0.15509863197803497,
 0.01046220026910305,
 0.427708238363266,
 0.3316727578639984,
 -0.5873403549194336,
 0.3590944707393646,
 0.2130666971206665,
 -0.05240130424499512,
 0.29795998334884644,
 -0.4877340793609619,
 0.5052562355995178,
 0.7285388708114624,
 0.904732346534729,
 0.17174264788627625,
 0.13098189234733582,
 -0.03580152243375778,
 -0.6401527523994446,
 0.01988145336508751,
 -0.30945152044296265,
 0.577164351940155,
 -0.2656308114528656,
 -0.6978050470352173,
 0.27843818068504333,
 0.1271088570356369,
 -0.23716838657855988,
 0.47018858790397644,
 0.09148619323968887,
 -0.09485956281423569,
 -0.3069428503513336,
 0.36983007192611694,
 -0.25263476371765137,
 0.2255743443965912,
 -0.283195823431015,
 -0.09026896953582764,
 -0.805753767490387,
 -0.4651011824

In [13]:
result = collection.search(
        data = [query_embedding], 
        anns_field = EMBEDDING_FIELD_NAME, 
        param = index_params, 
        limit = 100, 
        output_fields = ['complaint', 'soap', 'test_name', 'label'] # 보고 싶은 필드 명시
    )

In [163]:
result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
        
result_list

[{'id': 452150165267830897,
  'distance': 92.6534194946289,
  'entity': {'complaint': '상태문의',
   'soap': ') 어제 진료 보고 새빨간 설사. 후 갈색 초록색 설사 지속중 다시 구토도 시작하고 밥 안먹습니다.',
   'label': 0}},
 {'id': 452150165267785213,
  'distance': 89.02757263183594,
  'entity': {'complaint': '흑변, 혈뇨 ',
   'soap': '병력 : 어제부터 혈뇨 보기 시작함 식욕도 떨어짐 물 굉장히 많이 먹음 검은색 설사 보기 시작함 신체검사 :',
   'label': 0}},
 {'id': 452150165267914749,
  'distance': 86.32197570800781,
  'entity': {'complaint': '',
   'soap': '구토는 안하는데 배를 아파함 설사를 아직 하고 있음 - 배변 형태가 나오다가 나중에는 뚝뚝 떨어짐 배에 힘을 주고 끙끙거리는 증상이 심함 물을 먹고 나서 혹은 오줌 살 때 아파함 밥을 안먹어서 쌀 미음 만들어서 먹이고 계심 (스스로 먹지는 않음)',
   'label': 1}},
 {'id': 452150165267839769,
  'distance': 85.12216186523438,
  'entity': {'complaint': '배빵빵하고 숨을 못쉽니다.',
   'soap': '현재 배가 빵빵하고 숨을 못쉽니다. 물은 마시는데 밥은 잘 안먹습니다. 어제부터 대소변은 못보고 있습니다. 숨을 제대로 못쉬는데 왜이런가요? -> 병원내원하세요',
   'label': 0}},
 {'id': 452150165267884799,
  'distance': 84.59149169921875,
  'entity': {'complaint': '진료예약 ',
   'soap': '항생제먹으면 설사 지속 설사관련진료 이후 설사 잡히면 외과수술진

In [204]:
label_list = []

for i in range(len(result_list)):
    if result_list[i]['distance'] > 80:
        label_list.append(result_list[i]['entity']['label'])

print(len(label_list), np.array(label_list).sum())

9 6
